In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=40,
    remove_stop_words=False
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10565.55it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5901.65it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3473675.75it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11415.21it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5965.63it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 918840.55it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9674.71it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10a2f50b8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x109fb2438>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
padding_callback = mz.dataloader.callbacks.BasicPadding()

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.KNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

KNRM(
  (embedding): Embedding(30058, 100)
  (kernels): ModuleList(
    (0): GaussianKernel()
    (1): GaussianKernel()
    (2): GaussianKernel()
    (3): GaussianKernel()
    (4): GaussianKernel()
    (5): GaussianKernel()
    (6): GaussianKernel()
    (7): GaussianKernel()
    (8): GaussianKernel()
    (9): GaussianKernel()
    (10): GaussianKernel()
    (11): GaussianKernel()
    (12): GaussianKernel()
    (13): GaussianKernel()
    (14): GaussianKernel()
    (15): GaussianKernel()
    (16): GaussianKernel()
    (17): GaussianKernel()
    (18): GaussianKernel()
    (19): GaussianKernel()
    (20): GaussianKernel()
  )
  (out): Linear(in_features=21, out_features=1, bias=True)
)
Trainable params:  3005822


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-255 Loss-0.840]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5374 - normalized_discounted_cumulative_gain@5(0.0): 0.5928 - mean_average_precision(0.0): 0.5659



[Iter-510 Loss-0.464]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5272 - normalized_discounted_cumulative_gain@5(0.0): 0.5976 - mean_average_precision(0.0): 0.5617



[Iter-765 Loss-0.249]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5329 - normalized_discounted_cumulative_gain@5(0.0): 0.5999 - mean_average_precision(0.0): 0.5643



[Iter-1020 Loss-0.115]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5182 - normalized_discounted_cumulative_gain@5(0.0): 0.5931 - mean_average_precision(0.0): 0.5529



[Iter-1275 Loss-0.055]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5199 - normalized_discounted_cumulative_gain@5(0.0): 0.5864 - mean_average_precision(0.0): 0.5467



[Iter-1530 Loss-0.028]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5322 - normalized_discounted_cumulative_gain@5(0.0): 0.5956 - mean_average_precision(0.0): 0.5579



[Iter-1785 Loss-0.015]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5407 - normalized_discounted_cumulative_gain@5(0.0): 0.6016 - mean_average_precision(0.0): 0.5606



[Iter-2040 Loss-0.008]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.537 - normalized_discounted_cumulative_gain@5(0.0): 0.5925 - mean_average_precision(0.0): 0.5565



[Iter-2295 Loss-0.007]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5331 - normalized_discounted_cumulative_gain@5(0.0): 0.5887 - mean_average_precision(0.0): 0.5524



[Iter-2550 Loss-0.006]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5327 - normalized_discounted_cumulative_gain@5(0.0): 0.5929 - mean_average_precision(0.0): 0.5537

Cost time: 543.3827939033508s
